In [1]:
import subprocess

def blame(repo_root, file):
    return subprocess.run(["git", "blame", file],
       capture_output=True,
       cwd=repo_root
    ).stdout.decode("utf8")

def git_ls_files(repo_root, dir=None):
    args = ["git", "ls-files"]
    if dir is not None:
        args.append(dir) 
        
    p = subprocess.run(args,
       capture_output=True,
       cwd=repo_root
    )
    
    if p.returncode != 0:
        return p.stderr.decode("utf8")
    return p.stdout.decode("utf8").split("\n")


In [14]:
import re, pathlib, os.path as path

limit = 10
root_dir = path.dirname(pathlib.Path().absolute())
repo_names = [
    "storj",
]
author_regex = re.compile("\w+\s+\(([\w ]+)\s*?\d{4,}")

author_line_counts = {}

for name in repo_names:
    repo_root = path.join(root_dir, name)
    files = git_ls_files(repo_root=repo_root)
    for i, f in enumerate(files):
        if f != "" and i < limit:
            print(f)
            blame_lines = blame(repo_root, f)
            for line in blame_lines.split("\n"):
                if line == "":
                    continue
                    
                match = author_regex.match(line)
                if match is None:
                    print("no match: %s" % line)
                continue
            
            author = match.group(1)
            if author is None:
                print("no author: %s" % line)
                continue
                
            # print(author)
            if author in author_line_counts:
                author_line_counts[author] += 1
            else:
                author_line_counts[author] = 1
                
author_line_counts

.clabot
.dockerignore
.github/CODEOWNERS
.github/pull_request_template.md
.gitignore
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400  1) # Mac OS X files
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400  2) .DS_Store
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400  3) 
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400  4) # Binaries for programs and plugins
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400  5) *.exe
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400  6) *.dll
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400  7) *.so
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400  8) *.dylib
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400 10) 
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400 11) # Test binary, build with `go test -c`
no match: ^250b36f27 (Dennis Coyle    2018-04-06 12:32:34 -0400 12) *.test
no match: ^250b36f27 (Dennis Co

{'Dennis Coyle     ': 2,
 'Matt Robinson ': 1,
 'JT Olio ': 2,
 'JT Olio             ': 1,
 'Michal Niewrzal ': 1,
 'Ethan Adams ': 1,
 'Brandon Iglesias ': 1,
 'Stefan Benten         ': 1}

In [3]:
author_line_counts
                    

{'Dennis': 1, 'Matt': 1}

In [4]:
from github import Github

apikey = open("github.apikey")
               
g = Github()
org = g.get_organization("storj")
repos = org.get_repos()
page = repos.get_page(0)

[(r.name, r.created_at) for r in page]


[('pushy-cat', datetime.datetime(2014, 4, 21, 23, 49, 38)),
 ('storjshare-gui', datetime.datetime(2014, 7, 15, 20, 4, 18)),
 ('bridge-gui', datetime.datetime(2015, 12, 3, 19, 18, 7)),
 ('autobin', datetime.datetime(2015, 12, 28, 16, 13, 20)),
 ('bridge', datetime.datetime(2016, 1, 7, 16, 46, 33)),
 ('core', datetime.datetime(2016, 2, 2, 16, 10, 31)),
 ('storj.io', datetime.datetime(2016, 3, 28, 19, 59, 15)),
 ('pullypusher', datetime.datetime(2016, 3, 31, 18, 20, 34)),
 ('statusify', datetime.datetime(2016, 3, 31, 21, 27, 26)),
 ('data-api', datetime.datetime(2016, 4, 13, 17, 8, 53)),
 ('chef-storj', datetime.datetime(2016, 5, 25, 14, 3, 4)),
 ('sips', datetime.datetime(2016, 6, 24, 14, 47, 14)),
 ('storj-python-sdk', datetime.datetime(2016, 8, 11, 14, 24, 21)),
 ('heroku-addon', datetime.datetime(2016, 8, 24, 10, 55, 48)),
 ('heroku-email-service', datetime.datetime(2016, 8, 24, 10, 58, 46)),
 ('kfs', datetime.datetime(2016, 8, 26, 19, 38, 3)),
 ('storj-node-heroku-example', datetime.